# Introduction à l'apprentissage automatique: TP5 - Exercice 3 - <font color=red> CORRECTION </font>

<br>

### Prédiction de la qualité de vins

Le dataset suivant:

https://www.openml.org/d/40691

fournit la description de 1599 vins rouges: 12 mesures physico-chimiques ainsi qu'un critère qualitatif donné comme une note entre 3 et 8 (plus haute est la note, meilleur est le vin).

Remarquez que les 12 caractéristiques doivent être normalisées.

La cellule suivante charge les données, construit des bases d'apprentissage et de test, et normalise les caractéristiques.


In [ ]:
from sklearn import datasets, metrics, neural_network, svm, model_selection, preprocessing
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

# chargement des données
X_wine, y_wine = datasets.fetch_openml('wine-quality-red', return_X_y=True, as_frame=False, parser='auto')

n_samples = len(X_wine)
print("nombre total d'observations (apprentissage + test): %d" % n_samples)

n_features = len(X_wine[0])
print("nombre de caractéristiques par observation: %d" % n_features)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X_wine, y_wine, test_size=0.3, random_state=1)

print("nombre d'observations dans la base d'apprentissage: %d" %len(X_train))
print("nombre d'observations dans la base de test: %d" %len(X_test))

print("\n Cinq premières observations de X_train:")
print(X_train[:5,:])
print("\n et classes associées:")
print(y_train[:5])

# normalisation:
X_train_n = preprocessing.StandardScaler().fit_transform(X_train)
X_test_n = preprocessing.StandardScaler().fit(X_train).transform(X_test)


Deux remarques:

- Certaines caractéristiques semblent corrélées (on pourrait le vérifier en traçant des graphiques et en calculant des coefficients de corrélation comme dans le cours d'analyse de données): il serait sans doute pertinent de sélectionner des caractéristiques ou de réduire la dimension des observations.

- Nous allons envisager ce problème comme un problème de classification à 6 classes (les notes de 3 à 8). Néanmoins, il ne semblerait pas absurde de l'envisager comme un problème de régression.


<br>

Proposez des prédicteurs de la qualité en fonction des 12 mesures physico-chimiques. Vous explorerez les machines à vecteurs supports et les perceptrons multicouches, dont vous fixerez les hyperparamètres par _grid search_ et validation croisée.

<br>

Comparez vos résultats à ceux reportés ici:
https://www.openml.org/t/146217
(en vous demandant si les valeurs sont bien comparables)

In [ ]:
# non demandé
#############
# visualisation des scatter-plot d'une variable contre une autre
# on constate que certaines variables sont bien liées à d'autres
import pandas
import seaborn
data = datasets.fetch_openml('wine-quality-red', return_X_y=False, as_frame=True, parser='auto')
data_df = pandas.DataFrame(data.data, columns=data.feature_names)
seaborn.pairplot(data_df)
plt.show()

In [ ]:
# choix de C et gamma pour SVM et noyau gaussien  
C_range=10**(np.arange(-3.,3.5,.5))   
gamma_range=10**(np.arange(-3,3.5,.5))
parameters = { 'C':C_range, 'gamma':gamma_range }
SVM = svm.SVC(kernel='rbf')

gridsearch_SVM = model_selection.GridSearchCV(SVM, parameters,n_jobs=-1)

%time gridsearch_SVM.fit(X_train_n,y_train)
print("Avec normalisation, meilleur estimateur trouvé:")
print(gridsearch_SVM.best_estimator_)


In [ ]:
# inutile de ré-entraîner: par défaut gridsearch.fit finit par l'entraînement du meilleur modèle sélectionné par validation croisée
# SVM=svm.SVC(kernel='rbf',C=3.16,gamma=0.32)  
# SVM.fit(X_train_n,y_train)
# print("score SVM %.3f" % SVM.score(X_test_n, y_test) )

print("score SVM %.3f" % gridsearch_SVM.score(X_test_n, y_test) )

metrics.confusion_matrix(y_test, gridsearch_SVM.predict(X_test_n)) 
# remarque: globalement les valeurs sont regroupées autour de la diagonale de la matrice de confusion
# (sauf pour les notes 3 et 8) 
# ce qui est bon signe: quand on se trompe de classe on ne décale que d'une ou deux notes

In [ ]:
alpha_range=10**(np.arange(-6.,0.,2))
n_neurons_range=((10,), (50,), (100,), (10,10), (50,50))
max_iter_range=(10, 50, 100, 200)
parameters={ 'alpha':alpha_range, 'hidden_layer_sizes':n_neurons_range, 'max_iter':max_iter_range}
MLP = neural_network.MLPClassifier(parameters, random_state=1)

gridsearch_MLP = model_selection.GridSearchCV(MLP, parameters,n_jobs=-1)

%time gridsearch_MLP.fit(X_train_n,y_train)
print("Avec normalisation, meilleur estimateur trouvé:")
print(gridsearch_MLP.best_estimator_)

In [ ]:
#MLP=neural_network.MLPClassifier(alpha=1e-2, hidden_layer_sizes=(50,50), max_iter=100, random_state=1, verbose=true)  
#MLP.fit(X_train_n,y_train)

print("score MLP %.3f" % gridsearch_MLP.score(X_test_n, y_test) )
metrics.confusion_matrix(y_test, gridsearch_MLP.predict(X_test_n)) 

<font color=red>

Les matrices de confusion montrent que lorsqu'on se trompe, c'est généralement en prédisant un score proche du "vrai" score.
    
La SVM semble fournir des résultats un peu meilleurs (score de 0.613 contre 0.571 sur la base test)
    
Remarquons qu'une telle valeur de score pour un problème de classification à 6 classes n'est pas si mauvaise: si on prédisait au hasard on obtiendrait un score de 1/6 = 0.16. Si on prédisait systématiquement la classe de plus grand cardinal (note 5:  681 observations sur 1599), le score serait 0.43. 
    
Cela nous met dans le haut du panier des méthodes rassemblées ici:
    https://www.openml.org/t/146217
(voir `Analysis` puis sélectionner `predictive accuracy`)
    
Attention néanmoins, le score de la page openml est un score de 10-fold cross validation. Il faudrait comparer au score de la cellule suivante (encore que je ne suis pas certain de la normalisation utilisée dans les résultats d'openml):

</font>

In [ ]:
from sklearn.model_selection import cross_val_score
X_wine, y_wine = datasets.fetch_openml('wine-quality-red', return_X_y=True,parser='auto')
X_wine_n = preprocessing.StandardScaler().fit_transform(X_wine)
SVM=svm.SVC(kernel='rbf',C=3.16,gamma=0.32)  
scores = cross_val_score(SVM, X_wine_n, y_wine, cv=10)
print(np.mean(scores))